In [19]:
import requests
import json
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import urllib
import datetime
import os
from pytz import timezone
import pytz
from pathlib import Path
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options

In [2]:
DCurl = "https://www.dukascopy.com/swiss/english/home/?utm_source=freeserv"

In [3]:
def validatePath(path):
    """
    Checks if path refers to a valid directory. If not an exception is raised.
    """
    
    try:
        os.path.isfile(filename)
    except:
        raise NotADirectoryError("The specified path does not exist or is invalid. The data will be collected and stored in the current working directory in the file: ")

In [16]:
def gmtToUtc(time):
    utcTime = time.astimezone(timezone("UTC"))
    return utcTime.strftime(fmt)

def currUtcTime():
    return datetime.datetime.now(timezone('UTC')).strftime(fmt)

def toUtc(time):
    tz = pytz.timezone('UTC')
    return tz.localize(time)

In [5]:
def getDukasTime(soup):
    """
    Returns the extracted time as given on the DukasCopy webpage from where the DukasCopy data is being extracted from.
    
    Parameters:
    
    soup (soup): The BeautifulSoup object containing the parsed HTML representation of the DukasCopy webpage.
    
    Returns:
    date_time_obj (Datetime): Datetime object representing the time on the page when the data was taken.
    """
    tdata = soup.find_all("span", {"id": "timeUpdate"})
    dtxt = tdata[0].text
    
    date_time_obj = datetime.datetime.strptime(dtxt, ' %a, %d %b %Y %H:%M:%S GMT')
    
    return date_time_obj

In [6]:
def updateData(NewFrame, filename):
    """
    Updates the pickle file specified bt "filepath" by appending new data from the "NewFrame" dataframe to it.
    
    Parameters:
    
    NewFrame (DataFrame): The dataframe which is to be appended to the end of the pickle file to update the pickle file with the new data.
    filepath (str): The filepath of the pickle file where new dataframe data is to be appended.
    
    """
    if (not filename.endswith(".pkl")):
        raise Exception("The filename specified must end with .pkl extension. The filename provided was: " + filename)
    
    if not os.path.isfile(filename):
        pd.to_pickle(NewFrame, filename)
        
    else:   
        df = pd.read_pickle(filename)
        df = df.append(NewFrame, sort=False)
        df.to_pickle(filename)

In [7]:
# Settings for headless chrome

options = Options()
options.headless = True
options.add_argument('--no-proxy-server')
options.add_argument("--proxy-server='direct://'")
options.add_argument("--proxy-bypass-list=*")

In [17]:
def getDukasCopyData(pathToChromeDriver):
    """
    Returns a DataFrame object containing the most recent Gold Data from the DukasCopy website.
    Scrapes the data from the DukasCopy website before formatting and returning the data as a pandas DataFrame.
    
    Parameters:
    pathToChromeDriver (str): Path to the chromedriver executable to be used to open web pages.
    
    Returns:
    DCDataFrame (DataFrame): A pandas DataFrame containing the most recent Gold Data from the DukasCopy website.
    dataTime (Datetime): Datetime object representing the time at which data was fetched.
    """
    browser = webdriver.Chrome(pathToChromeDriver, options=options)
    browser.get(DCurl)
    
    DChtml = browser.page_source
    soup = BeautifulSoup(DChtml, "lxml")
    
    # Finding the exact table with all the required data
    data = soup.find_all("table", {"id": "list"})
    
    # Creating and Modifying dataframe with the table 
    DCDataFrame = pd.read_html(str(data))[0]
    
    # Add a timestamp to the data
    dataTime = toUtc(getDukasTime(soup))
    DCDataFrame["timestamp"] = dataTime
    DCDataFrame = DCDataFrame.set_index("timestamp")
    
    # Filter dataframe only to Gold Data
    DCDataFrame = DCDataFrame[DCDataFrame["Live"] == "XAU/USD"]
    
    browser.quit()
    return DCDataFrame, dataTime

In [9]:
def fetchAndSaveData(chromeDriverPath, filepath="DukasGoldData.pkl"):
    """
    Main driver for the code - calls all methods to fetch and save gold data, and alert in case of surges.
    """
    currData, timestamp = getDukasCopyData(chromeDriverPath)
    updateData(currData, filepath)

In [14]:
def run(chromeDriverPath, filepath="DukasGoldData.pkl", collectionTime=900):
    """
    Fetches and saves the most recent gold data from DukasCopy at an interval of "collectionTime" seconds
    """
    while True:
        fetchAndSaveData(chromeDriverPath)
        time.sleep(collectionTime - 5)

In [ ]:
run("chromedriver")